In [4]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")

if not API_KEY:
    raise RuntimeError("Brak GOOGLE_MAPS_API_KEY w .env")

def transit_route(origin: str, destination: str):
    url = "https://maps.googleapis.com/maps/api/directions/json"

    params = {
        "origin": origin,
        "destination": destination,
        "mode": "transit",          # 🔑 komunikacja miejska
        # opcjonalnie możesz ograniczyć typy:
        # "transit_mode": "bus|subway|train|tram|rail",
        "key": API_KEY
    }

    r = requests.get(url, params=params)
    print("Status code:", r.status_code)
    data = r.json()
    print("Raw response status:", data.get("status"))

    if data.get("status") != "OK":
        raise RuntimeError(data.get("error_message", "Directions failed"))

    route = data["routes"][0]
    leg = route["legs"][0]

    print("\nOdległość:", leg["distance"]["text"])
    print("Czas:", leg["duration"]["text"])

    print("\nKroki trasy (środki transportu):")
    for i, step in enumerate(leg["steps"], start=1):
        travel_mode = step["travel_mode"]
        if travel_mode == "WALKING":
            print(f"{i}. PIESZO – {step['distance']['text']}")
        elif travel_mode == "TRANSIT":
            t = step["transit_details"]
            line = t["line"]
            vehicle_type = line["vehicle"]["type"]  # BUS / TRAM / SUBWAY / TRAIN / RAIL
            line_name = line.get("short_name") or line.get("name")
            num_stops = t["num_stops"]
            dep_stop = t["departure_stop"]["name"]
            arr_stop = t["arrival_stop"]["name"]
            print(
                f"{i}. {vehicle_type}: linia {line_name}, "
                f"z {dep_stop} do {arr_stop} ({num_stops} przystanków)"
            )
        else:
            print(f"{i}. {travel_mode}")

if __name__ == "__main__":
    # podmień na jakieś sensowne lokalne adresy
    transit_route("Łódź, Poland", "Warsaw, Poland")


Status code: 200
Raw response status: OK

Odległość: 118 km
Czas: 1 hour 55 mins

Kroki trasy (środki transportu):
1. BUS: linia FlixBus, z Łódź Plac Bronisława Sałacińskiego do Warszawa (2 przystanków)


In [5]:
transit_route("Kołowa 19/21, Łódź", "Obywatelska Nowe Sady, Łódź")

Status code: 200
Raw response status: OK

Odległość: 5.1 km
Czas: 28 mins

Kroki trasy (środki transportu):
1. PIESZO – 0.7 km
2. BUS: linia 69A, z Kilińskiego - Broniewskiego (0270) do Obywatelska - Nowe Sady (0654) (9 przystanków)
3. PIESZO – 0.2 km


In [6]:
def transit_routes_all(origin: str, destination: str) -> None:
    """Wypisz wszystkie dostępne trasy komunikacją miejską z A do B."""
    url = "https://maps.googleapis.com/maps/api/directions/json"

    params = {
        "origin": origin,
        "destination": destination,
        "mode": "transit",
        "alternatives": "true",  # 🔑 prosimy o alternatywne trasy
        "key": API_KEY,
    }

    r = requests.get(url, params=params)
    print("HTTP status code:", r.status_code)
    data = r.json()
    print("API response status:", data.get("status"))

    if data.get("status") != "OK":
        raise RuntimeError(data.get("error_message", "Directions failed"))

    routes = data.get("routes", [])
    if not routes:
        raise RuntimeError("Brak tras dla podanych parametrów.")

    print(f"\nZnaleziono {len(routes)} tras(y):\n")

    for route_idx, route in enumerate(routes, start=1):
        leg = route["legs"][0]

        print(f"================== TRASA {route_idx} ==================")
        print("Odległość:", leg["distance"]["text"])
        print("Czas:", leg["duration"]["text"])

        print("\nKroki trasy (środki transportu):")
        for i, step in enumerate(leg["steps"], start=1):
            travel_mode = step["travel_mode"]
            if travel_mode == "WALKING":
                print(f"{i}. PIESZO – {step['distance']['text']}")
            elif travel_mode == "TRANSIT":
                t = step["transit_details"]
                line = t["line"]
                vehicle_type = line["vehicle"][
                    "type"
                ]  # BUS / TRAM / SUBWAY / TRAIN / RAIL
                line_name = line.get("short_name") or line.get("name")
                num_stops = t["num_stops"]
                dep_stop = t["departure_stop"]["name"]
                arr_stop = t["arrival_stop"]["name"]
                print(
                    f"{i}. {vehicle_type}: linia {line_name}, "
                    f"z {dep_stop} do {arr_stop} ({num_stops} przystanków)"
                )
            else:
                print(f"{i}. {travel_mode}")

        print()  # pusta linia między trasami

In [7]:
transit_routes_all("Kołowa 19/21, Łódź", "Obywatelska Nowe Sady, Łódź")

HTTP status code: 200
API response status: OK

Znaleziono 6 tras(y):

================== TRASA 1 ==================
Odległość: 5.1 km
Czas: 28 mins

Kroki trasy (środki transportu):
1. PIESZO – 0.7 km
2. BUS: linia 69A, z Kilińskiego - Broniewskiego (0270) do Obywatelska - Nowe Sady (0654) (9 przystanków)
3. PIESZO – 0.2 km

================== TRASA 2 ==================
Odległość: 5.1 km
Czas: 28 mins

Kroki trasy (środki transportu):
1. PIESZO – 0.7 km
2. BUS: linia 69A, z Kilińskiego - Broniewskiego (0270) do Obywatelska - Nowe Sady (0654) (9 przystanków)
3. PIESZO – 0.2 km

================== TRASA 3 ==================
Odległość: 7.4 km
Czas: 39 mins

Kroki trasy (środki transportu):
1. PIESZO – 0.6 km
2. TRAM: linia 5, z Kilińskiego - Dąbrowskiego (0271) do Kilińskiego - Fabryczna (0282) (5 przystanków)
3. BUS: linia 55A, z Kilińskiego - Fabryczna (0282) do Obywatelska - Nowe Sady (0654) (11 przystanków)
4. PIESZO – 0.2 km

================== TRASA 4 ==================
Odległość: 7